In [1]:
import requests
import pandas as pd
import csv
from numpy.random.mtrand import randint
import time


In [2]:
def zip_codes():
    """method to extract and create a list with all zip codes"""
    filename = '../raw_data/Berlin Zip Codes - Sheet1.csv'
    zip_codes_list = []
    with open(filename, 'r') as csvfile:
        datareader = csv.reader(csvfile)
        for row in datareader:
            zip_codes_list.append(row[3:])
    return zip_codes_list[1:]


# list with all Berlin zipcodes
zip_code_list = zip_codes()

In [18]:
def api_scraper(coords_list):
    """ returns information for all venues in wolt, given a coordinates list"""

    headers = {
        'authority': 'restaurant-api.wolt.com',
        'method': 'GET',
        'path':
        '/v1/pages/restaurants?lat=52.516913793238444&lon=13.389984460880868',
        'scheme': 'https',
        'accept': 'application/json, text/plain, */*',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en,es;q=0.9',
        'app-language': 'en',
        'clientversionnumber': '1.6.151',
        'cookie':
        'ravelinDeviceId=rjs-f3df3331-247a-46ac-b626-ec1627b582a2; intercom-id-qwum5ehb=1531a790-2768-4fda-9d8f-41f5f73fe476; intercom-session-qwum5ehb=; _gcl_au=1.1.385640245.1637093317; _fbp=fb.1.1637093318113.856351982; afUserId=07a6dae1-f582-4784-9eb4-924f09ac098b-p; _gac_UA-56809017-2=1.1637093394.Cj0KCQiAys2MBhDOARIsAFf1D1cvfq1acQ1I1tnUILzwSp5QqyaEVycWm41K1fVzZgIvotYve_oDYVQaAoCWEALw_wcB; _gid=GA1.2.173313073.1638290842; AF_SYNC=1638312110265; ravelinSessionId=bb56d33d-f245-4653-ad41-e8136dd9f235; _ga=GA1.2.1935941802.1637093317; _ga_CP7Z2F7NFM=GS1.1.1638347246.30.1.1638347477.60',
        'origin': 'https://wolt.com',
        'platform': 'Web',
        'referer': 'https://wolt.com/',
        'sec-ch-ua':
        '" Not A;Brand";v="99", "Chromium";v="96", "Google Chrome";v="96"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': "macOS",
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'user-agent':
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.55 Safari/537.36',
        'w-wolt-session-id': '11394316-3220-4d4b-9676-96e1f9b407c9'
    }

    restaurants_list = []

    base_url = 'https://restaurant-api.wolt.com/v1/pages/restaurants'


    for coord in coords_list:
        params = {"lat": coord[0], "lon": coord[1]}
        response = requests.get(base_url, headers=headers, params=params).json()
        try:
            for resto in range(len(response['sections'][0]['items'])):
                resto_dict = {}
                resto_dict['restaurant_name'] = response["sections"][0]['items'][resto]['title']
                try:
                    resto_dict['avg_review_score'] = response['sections'][0]['items'][resto]['venue']['rating']['score']
                except Exception:
                    resto_dict['avg_review_score'] = None
                # reviews - TBU
                resto_dict['pricyness'] = response['sections'][0]['items'][resto]['venue']['price_range']
                # minimum_order_value - TBU
                resto_dict['delivery_fee'] = response['sections'][0]['items'][resto]['venue']['delivery_price'][1:]
                try:
                    resto_dict['type_of_cuisine'] = response['sections'][0]['items'][resto]['filtering']['filters'][0]["values"][0]
                except Exception:
                    resto_dict['type_of_cuisine'] = None
                # address - TBU
                resto_dict['street'] = response['sections'][0]['items'][resto]['venue']['address']
                # zip_code - TBU
                resto_dict['city_name'] = 'berlin'
                resto_dict['coordinates'] = response['sections'][0]['items'][resto]['venue']['location']
                resto_dict['latitude'] = response['sections'][0]['items'][resto]['venue']['location'][1]
                resto_dict['longitude'] = response['sections'][0]['items'][resto]['venue']['location'][0]
                resto_dict['link'] = response['sections'][0]['items'][resto]['link']['target']

                restaurants_list.append(resto_dict)
        except Exception as e:
            print(e)

        time.sleep(randint(1, 3))


    restaurants_df = pd.DataFrame(restaurants_list)

    return restaurants_df


In [16]:
wolt_df_clean = wolt_df.drop(columns=['coordinates']).drop_duplicates(subset=[
    'restaurant_name', 'pricyness', 'street', 'city_name',
    'latitude', 'longitude', 'link'], keep="first")
wolt_df_clean = wolt_df_clean.reset_index(drop=True)
wolt_df_clean['coordinates'] = wolt_df_clean.latitude.astype(
    str) + "," + wolt_df_clean.longitude.astype(str)
wolt_df_clean["database"] = "wolt"
wolt_df_clean = wolt_df_clean[[
    'restaurant_name', 'pricyness', 'delivery_fee',
    'street', 'city_name', 'latitude', 'longitude', 'coordinates', 'link',
    'database'
]]
# wolt_df_clean.to_csv("../geotracker/data/wolt_restaurants_from_api.csv", index=False)


In [39]:
tmp = pd.read_csv("../geotracker/data/all_data.csv").iloc[:,1:]
tmp = tmp[tmp.database == "wolt"]
tmp.head()

,restaurant_name,avg_review_score,reviews,type_of_cuisine,address,street,zip_code,city_name,coordinates,latitude,longitude,database,opening_hours
0,BunUp Mitte,7.8,NaN,vegetarian or vegan,"This is a virtual venue, 10178.0, Berlin",This is a virtual venue,10178.0,Berlin,"52.5238316,13.3985704",52.523832,13.398570,wolt,NaN
1,Machiavelli,8.4,NaN,european,"Albrechtstr.13 Berlin, 10117.0, Berlin",Albrechtstr.13 Berlin,10117.0,Berlin,"52.52130280012305,13.385180090564926",52.521303,13.385180,wolt,NaN
2,Pizza Peppino,NaN,NaN,european,"Tucholskystraße 13, 10117.0, Berlin",Tucholskystraße 13,10117.0,Berlin,"52.52339332586772,13.391764043640274",52.523393,13.391764,wolt,NaN
3,Fresh's,7.8,NaN,poke,"This is a virtual venue, 10178.0, Berlin",This is a virtual venue,10178.0,Berlin,"52.5238316,13.3985704",52.523832,13.398570,wolt,NaN
4,Bombay,8.4,NaN,middle eastern,"Friedrichstraße 106c, 10117.0, Berlin",Friedrichstraße 106c,10117.0,Berlin,"52.5232743,13.3884602",52.523274,13.388460,wolt,NaN


In [37]:
tmp2 = pd.read_csv('../geotracker/data/wolt_restaurants_from_api.csv')
tmp2.head()

,restaurant_name,pricyness,delivery_fee,street,city_name,latitude,longitude,coordinates,link,database
0,Burgermeister Kottbusser Tor,1,2.9,Skalitzer Str. 136,berlin,52.499503,13.418597,"52.49950319999998,13.418596600000013",https://wolt.com/en/deu/berlin/restaurant/burg...,wolt
1,Pizza Meister,1,2.9,Zossener Straße 12,berlin,52.492279,13.394353,"52.4922794,13.3943531",https://wolt.com/en/deu/berlin/restaurant/pizz...,wolt
2,Burgermann,1,2.9,Schönhauser Allee 173,berlin,52.532460,13.412156,"52.532460231915074,13.412155674891162",https://wolt.com/en/deu/berlin/restaurant/burg...,wolt
3,Bergmann Burger by BeefMan,1,2.9,Bergmannstraße 112,berlin,52.490227,13.387341,"52.4902266,13.3873411",https://wolt.com/en/deu/berlin/restaurant/berg...,wolt
4,Pizza Noi,1,2.9,Potsdamer Straße 112,berlin,52.500574,13.363153,"52.50057444327202,13.363152750293107",https://wolt.com/en/deu/berlin/restaurant/pizz...,wolt


In [49]:
new_data_withcuisine = pd.merge(tmp2,
         tmp[["restaurant_name", "latitude", "longitude", "type_of_cuisine", "avg_review_score"]],
         how="left",
         on=["restaurant_name", "latitude", "longitude"])
new_data_withcuisine = new_data_withcuisine.drop_duplicates()
new_data_withcuisine.to_csv(
    "../geotracker/data/wolt_restaurants_from_api_2.csv", index=False)


In [57]:
wolt_for_web = new_data_withcuisine[["latitude", "longitude"]]
wolt_for_web = wolt_for_web.drop_duplicates()

wolt_for_web = wolt_for_web.rename(columns={
    "latitude": "lat",
    "longitude": 'lon'
})
wolt_for_web.to_csv("../geotracker/data/wolt_restaurants_from_api_latlon.csv",
                    index=False)
